In [ ]:
# Uncomment to download

# import kagglehub
# path = kagglehub.dataset_download("marlesson/myanimelist-dataset-animes-profiles-reviews")
# print("Path to dataset files:", path)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

animes_df = pd.read_csv("original_data/animes.csv")
profiles_df = pd.read_csv("original_data/profiles.csv")
reviews_df = pd.read_csv("original_data/reviews.csv")

In [ ]:
# keep uid and title columns, change uid to a_id
cleared_anime_df = animes_df[["uid", "title"]]
cleared_anime_df.rename(columns={"uid": "a_id"}, inplace=True)

# keep profile column, make a new column named u_id, make profile unique, keep u_id as index
cleared_profile_df = profiles_df[["profile"]]
cleared_profile_df = cleared_profile_df.drop_duplicates(subset='profile')
cleared_profile_df['u_id'] = range(0, len(cleared_profile_df))


# keep profile anime_uid and score columns, change anime_uid to a_id
cleared_reviews_df = reviews_df[["profile", "anime_uid", "score"]]
cleared_reviews_df.rename(columns={"anime_uid": "a_id"}, inplace=True)

In [ ]:
# merge cleared_reviews_df with cleared_profile_df using profile column
merged_review_profile_df = cleared_reviews_df.merge(cleared_profile_df, on="profile")    

# merge cleared_anime_df with merged_review_profile_df using a_id column
merged_review_profile_anime_df = merged_review_profile_df.merge(cleared_anime_df, on="a_id")

# at some point, we create duplicates. Removes duplicates
merged_review_profile_anime_df = merged_review_profile_anime_df.drop_duplicates()

In [ ]:
# reorder columns
final_df = merged_review_profile_anime_df[["u_id", "a_id", "profile", "title", "score"]]

# create id, and set it as index
final_df["id"] = range(0, len(final_df))
final_df.set_index("id", inplace=True)

# remove index name
final_df.index.name = None

In [ ]:
# only keep the users with at least 4 reviews
final_df = final_df[final_df.groupby('u_id')['u_id'].transform('count') > 3]

# only keep the animes with at least 11 reviews
final_df = final_df[final_df.groupby('a_id')['a_id'].transform('count') > 10]
